# evaluating entities

In [2]:
from evaluate import *
pred_file='../dataset/spert/predictions/intra-sent.json'
source_file='../dataset/spert/test_combined.json'
out_dir='../dataset/spert/predictions/BRAT'
ann_path='../dataset/BRAT/test'
#offset_key,start_key,end_key="tokens",'start','end'
offset_key,start_key,end_key="token_offsets",0,1
json2brat(source_file,ann_path,offset_key,start_key,end_key,pred_file,out_dir)

5485505111
E221	Problem:T221 Assertion:T222
4959313770
E109	Drug:T109
4959313770
E113	Drug:T113
6073394145
E72	Problem:T72 Assertion:T73


In [3]:
# summarizing entity performance

from glob import glob
exps=['spert']
for exp in exps:
        scores={
        "Drug,Trigger":[0,0,0],
        "Problem,Trigger":[0,0,0],
        "Problem,Assertion":[0,0,0],
        "Problem,Change":[0,0,0],
        "Problem,Severity":[0,0,0],
        "Problem,Anatomy":[0,0,0],
        "Problem,Characteristics":[0,0,0],
        "Problem,Duration":[0,0,0],
        "Problem,Frequency":[0,0,0],
        }

        for file in glob(f'../dataset/{exp}/predictions/*.csv'):
            if exp in file and '_detailed.csv' in file:
                lines=open(file).read().split('\n')[1:]
                out_ids=[]
                for line in lines:
                    if 'Problem,Trigger' in line and float(line.split(',')[-1])==0.5:
                        out_ids.append(line.split(',')[0])
                    
                for line in lines:
                    if line:
                        words=line.split(',')[1:]
                        key=','.join(words[:2])
                        if key not in scores:
                            #print(key)
                            continue
                        if any([id in line for id in out_ids]):
                            #print(line)
                            continue
                        if words[3]!='0':
                            for i in range(3):
                                scores[key][i]+=int(words[i+3])
        scores['Overall,-']=[0,0,0]
        for i in range(3):
            scores['Overall,-'][i]=sum([v[i] for k,v in scores.items()])

        with open(f'../performance/entity_{exp}.csv','w') as f:
            f.write('event,Argument,NT,NP,TP,P,R,F1\n')
            for key in scores:     
                f.write(key+','+','.join([str(s) for s in scores[key]]))
                P=scores[key][2]*100.0/scores[key][1] if scores[key][1] else 0
                R=scores[key][2]*100.0/scores[key][0] if scores[key][0] else 0
                F1=2*P*R/(P+R) if R+P else 0
                f.write(f",{round(P,1)},{round(R,1)},{round(F1,1)}\n")

# evaluating relations

In [3]:
# converting from plmarker to spert json
from evaluate import *
source_file='../dataset/plmarker/test_plmarker.jsonl'
exp='R15_re'
pred_ent_file=f'/home/velvinfu/code/PL-Marker/experiments/{exp}/ner_pred_results.json'
pred_re_file=f'/home/velvinfu/code/PL-Marker/experiments/{exp}/re_pred_results.json'
out_file=f'../dataset/plmarker/predictions/intra_re.json'

plmarkerPred2json(source_file,pred_ent_file,pred_re_file,out_file)

In [5]:
from evaluate import *
model='plmarker'#{spert}
exp='intra-re'
pred_file=f'../dataset/{model}/predictions/{exp}.json'
source_file='../dataset/spert/test_combined.json'
out_file=f'../performance/{model}_{exp}.csv'
error_file=f'../performance/{model}_{exp}_error.json'
score_relations(pred_file,source_file,out_file,error_file)